In [1]:
import os
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt

In [5]:
data_path = 'C:/Users/ppras/ML/HRA/Flask-master/CIFAR-10-image-classification/sample_videos'

In [6]:
categoeries = os.listdir(data_path)
labels = [i for i in range(len(categoeries))]
label_dict = dict(zip(categoeries, labels))

In [15]:
print(label_dict)
print(categoeries)
print(labels)

{'Bowling': 0, 'Boxing': 1, 'Brushing_Teeth': 2, 'cricket': 3, 'Drumming': 4, 'Fencing': 5, 'Golf': 6, 'Handstand': 7, 'tennis': 8, 'typing': 9}
['Bowling', 'Boxing', 'Brushing_Teeth', 'cricket', 'Drumming', 'Fencing', 'Golf', 'Handstand', 'tennis', 'typing']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [16]:
root_path = 'C:/Users/ppras/ML/HRA/frames' # Path for writing frames from videos

In [54]:
data=[] # Appending images as array in the list
target=[] #Target list to make prediction

In [55]:
folder_count = 0
img_size=32

In [56]:
for items in categoeries: #looping over folders in categoeries
    path = os.path.join(root_path, items) 
    folder_path = os.path.join(data_path,items)
    img_names = os.listdir(folder_path) # Listing all files inside the directory
    os.mkdir(path)
    img_count= 0
    folder_count +=1
    
    for img_name in img_names:
            cap = cv2.VideoCapture(os.path.join(folder_path, img_name)) # Capcturing video and dividing it by frames
            success, image = cap.read()
            while success:
                success, image = cap.read()
                if not success:
                    break
                
                
                #gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # Converting coloured image to greyScale
                #norm = cv2.normalize(gray_image, None, alpha=0,beta=200, norm_type=cv2.NORM_MINMAX) # Normalize image
                #resized_up = cv2.resize(image, down_points, interpolation= cv2.INTER_LINEAR) # Resizing image
                resized=cv2.resize(image,(img_size,img_size))
                cv2.imwrite(path + '/' + items +str(img_count) + '.jpg' , resized) # Writing image into to specified class with its class name 
                data.append(resized)
                target.append(label_dict[items])
                
                img_count += 1

In [57]:
def normalize(x):
    x = np.array(data)
    x = x.astype('float32')
    x = x/255.0
    return x

In [58]:
data_x = normalize(data)

In [70]:
target=np.array(target)

In [71]:
target = target.reshape(-1,)
target[:5]

array([0, 0, 0, 0, 0])

In [72]:
classes = ['Bowling', 'Boxing', 'Brushing_Teeth', 'cricket', 'Drumming', 'Fencing', 'Golf', 'Handstand', 'tennis', 'typing']

In [73]:
data_x.shape

(8466, 32, 32, 3)

In [75]:
target.shape

(8466,)

In [76]:
from sklearn.model_selection import train_test_split
X_train ,X_test ,y_train ,y_test =train_test_split(data_x,target,test_size=0.3)

In [77]:
print("X Train shape :", X_train.shape)
print("X Test shape :", X_test.shape)
print("y Train shape :", y_train.shape)
print("y test shape :", y_test.shape)

X Train shape : (5926, 32, 32, 3)
X Test shape : (2540, 32, 32, 3)
y Train shape : (5926,)
y test shape : (2540,)


In [78]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np

In [79]:
ann = models.Sequential([
        layers.Flatten(input_shape=(32,32,3)),
        layers.Dense(3000, activation='relu'),
        layers.Dense(1000, activation='relu'),
        layers.Dense(10, activation='softmax')    
    ])

ann.compile(optimizer='SGD',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

ann.fit(X_train, y_train, epochs=5)

Epoch 1/5
186/186 [==============================] - 14s 71ms/step - loss: 0.5109 - accuracy: 0.9310
Epoch 2/5
186/186 [==============================] - 13s 70ms/step - loss: 0.0797 - accuracy: 0.9998
Epoch 3/5
186/186 [==============================] - 13s 67ms/step - loss: 0.0377 - accuracy: 1.0000
Epoch 4/5
186/186 [==============================] - 13s 70ms/step - loss: 0.0235 - accuracy: 1.0000
Epoch 5/5
186/186 [==============================] - 13s 70ms/step - loss: 0.0167 - accuracy: 1.0000


In [81]:
from sklearn.metrics import confusion_matrix , classification_report
y_pred = ann.predict(X_test)
y_pred_classes = [np.argmax(element) for element in y_pred]

print("Classification Report: \n", classification_report(y_test, y_pred_classes))

Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       190
           1       1.00      1.00      1.00       216
           2       1.00      1.00      1.00       320
           3       1.00      1.00      1.00       183
           4       1.00      1.00      1.00       434
           5       1.00      1.00      1.00       185
           6       1.00      1.00      1.00       313
           7       1.00      1.00      1.00       146
           8       1.00      1.00      1.00       285
           9       1.00      1.00      1.00       268

    accuracy                           1.00      2540
   macro avg       1.00      1.00      1.00      2540
weighted avg       1.00      1.00      1.00      2540



In [82]:
cnn = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

In [83]:
cnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [84]:
cnn.fit(X_train, y_train, epochs=10)

Epoch 1/10
186/186 [==============================] - 8s 40ms/step - loss: 0.3113 - accuracy: 0.9178
Epoch 2/10
186/186 [==============================] - 7s 37ms/step - loss: 0.0025 - accuracy: 1.0000
Epoch 3/10
186/186 [==============================] - 7s 38ms/step - loss: 6.4350e-04 - accuracy: 1.0000
Epoch 4/10
186/186 [==============================] - 7s 38ms/step - loss: 3.1159e-04 - accuracy: 1.0000
Epoch 5/10
186/186 [==============================] - 7s 38ms/step - loss: 1.8228e-04 - accuracy: 1.0000
Epoch 6/10
186/186 [==============================] - 7s 37ms/step - loss: 1.1118e-04 - accuracy: 1.0000
Epoch 7/10
186/186 [==============================] - 7s 38ms/step - loss: 7.9606e-05 - accuracy: 1.0000
Epoch 8/10
186/186 [==============================] - 7s 37ms/step - loss: 5.7849e-05 - accuracy: 1.0000
Epoch 9/10
186/186 [==============================] - 7s 37ms/step - loss: 4.3654e-05 - accuracy: 1.0000
Epoch 10/10
186/186 [==============================] - 7s 37ms/

In [98]:
y_pred = cnn.predict(X_test)
y_pred[:5]

array([[9.99999523e-01, 5.25387520e-07, 2.24396168e-10, 1.09263857e-12,
        5.80592172e-11, 2.49206322e-10, 3.43901035e-14, 5.91906895e-14,
        1.70801262e-09, 2.79838352e-09],
       [1.28817953e-17, 3.97446621e-15, 1.96739796e-23, 6.35131443e-18,
        1.44424715e-11, 1.00000000e+00, 7.07060910e-25, 2.26767608e-16,
        6.57532099e-23, 7.10026569e-12],
       [1.70421856e-07, 2.13354556e-10, 3.34814260e-10, 6.36691269e-08,
        9.99996781e-01, 1.52295968e-10, 1.29803834e-06, 1.73963463e-06,
        4.70227131e-17, 8.36497893e-09],
       [1.25557321e-07, 1.17400667e-08, 1.91344760e-14, 6.46079146e-09,
        2.26728844e-06, 9.99997497e-01, 4.12358576e-14, 8.82697009e-08,
        3.16231606e-13, 4.04670164e-09],
       [3.08215875e-09, 5.64331137e-10, 3.22233823e-10, 2.28272370e-06,
        4.93910690e-08, 7.59717403e-15, 9.99997735e-01, 1.24100641e-08,
        1.15403945e-11, 1.68082312e-10]], dtype=float32)

In [99]:
y_classes = [np.argmax(element) for element in y_pred]
y_classes[:5]

[0, 5, 4, 5, 6]

In [100]:
y_test[:5]

array([0, 5, 4, 5, 6])

In [109]:
import matplotlib.pyplot as plt

In [110]:
classes[y_classes[3]]

'Fencing'

In [111]:
classes[y_classes[9]]

'tennis'